# Building an Intelligent Customer Support Agent with LangGraph

## Overview
This tutorial demonstrates how to create an intelligent customer support agent using LangGraph, a powerful tool for building complex language model workflows. The agent is designed to categorize customer queries, analyze sentiment, and provide appropriate responses or escalate issues when necessary.

## Motivation
In today's fast-paced business environment, efficient and accurate customer support is crucial. Automating the initial stages of customer interaction can significantly reduce response times and improve overall customer satisfaction. This project aims to showcase how advanced language models and graph-based workflows can be combined to create a sophisticated support system that can handle a variety of customer inquiries.

## Key Components
1. **State Management**: Using TypedDict to define and manage the state of each customer interaction.
2. **Query Categorization**: Classifying customer queries into Technical, Billing, or General categories.
3. **Sentiment Analysis**: Determining the emotional tone of customer queries.
4. **Response Generation**: Creating appropriate responses based on the query category and sentiment.
5. **Escalation Mechanism**: Automatically escalating queries with negative sentiment to human agents.
6. **Workflow Graph**: Utilizing LangGraph to create a flexible and extensible workflow.

## Method Details
1. **Initialization**: Set up the environment and import necessary libraries.
2. **State Definition**: Create a structure to hold query information, category, sentiment, and response.
3. **Node Functions**: Implement separate functions for categorization, sentiment analysis, and response generation.
4. **Graph Construction**: Use StateGraph to define the workflow, adding nodes and edges to represent the support process.
5. **Conditional Routing**: Implement logic to route queries based on their category and sentiment.
6. **Workflow Compilation**: Compile the graph into an executable application.
7. **Execution**: Process customer queries through the workflow and retrieve results.

## Conclusion
This tutorial demonstrates the power and flexibility of LangGraph in creating complex, AI-driven workflows. By combining natural language processing capabilities with a structured graph-based approach, we've created a customer support agent that can efficiently handle a wide range of queries. This system can be further extended and customized to meet specific business needs, potentially integrating with existing customer support tools and databases for even more sophisticated interactions.

The approach showcased here has broad applications beyond customer support, illustrating how language models can be effectively orchestrated to solve complex, multi-step problems in various domains.

## Install dependancies to run this tutorial

In [ ]:
# Install necessary packages
# !pip install -q langgraph langchain-core langchain-openai python-dotenv ipython


## Import necessary libraries


In [1]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod
from dotenv import load_dotenv
import os

# Load environment variables and set OpenAI API key
# load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
from Completed.Import_LLM import llm

c:\Users\A0262\Downloads\LearnAgents\GenAI_Agents\all_agents_tutorials\Completed\Import_LLM.py:19: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


## Define State Structure

We define a `State` class to hold the query, category, sentiment, and response for each customer interaction.

In [3]:
# Print key characteristics of the LLM for debugging and sharing with support tools

def print_llm_characteristics(llm_instance):
    print("LLM Class:", llm_instance.__class__.__name__)
    print("Model Name:", getattr(llm_instance, "model_name", "N/A"))
    print("Temperature:", getattr(llm_instance, "temperature", "N/A"))
    print("Max Tokens:", getattr(llm_instance, "max_tokens", "N/A"))
    print("API Key Present:", hasattr(llm_instance, "openai_api_key") and bool(getattr(llm_instance, "openai_api_key", None)))
    print("Azure Endpoint:", getattr(llm_instance, "azure_endpoint", "N/A"))
    print("API Version:", getattr(llm_instance, "openai_api_version", "N/A"))
    print("API Type:", getattr(llm_instance, "openai_api_type", "N/A"))
    print("Deployment Name:", getattr(llm_instance, "deployment_name", "N/A"))
print_llm_characteristics(llm)

LLM Class: AzureChatOpenAI
Model Name: gpt-3.5-turbo
Temperature: 0.2
Max Tokens: 1000
API Key Present: True
Azure Endpoint: https://axtria-institute-ds.openai.azure.com
API Version: 2024-12-01-preview
API Type: azure
Deployment Name: gpt-4o-mini


In [4]:
class State(TypedDict):
    query: str
    category: str
    sentiment: str
    response: str

## Define Node Functions

These functions represent the different stages of processing a customer query.

In [8]:
def categorize(state: State) -> State:
    """Categorize the customer query into Technical, Billing, or General."""
    prompt = ChatPromptTemplate.from_template(
        "Categorize the following customer query into one of these categories: "
        "Technical, Billing, General. Query: {query}"
    )
    chain = prompt | llm
    category = chain.invoke({"query": state["query"]}).content
    return {"category": category}

def analyze_sentiment(state: State) -> State:
    """Analyze the sentiment of the customer query as Positive, Neutral, or Negative."""
    prompt = ChatPromptTemplate.from_template(
        "Analyze the sentiment of the following customer query. "
        "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"
    )
    chain = prompt | llm
    sentiment = chain.invoke({"query": state["query"]}).content
    return {"sentiment": sentiment}

def handle_technical(state: State) -> State:
    """Provide a technical support response to the query."""
    prompt = ChatPromptTemplate.from_template(
        "Provide a technical support response to the following query: {query}"
    )
    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content
    return {"response": response}

def handle_billing(state: State) -> State:
    """Provide a billing support response to the query."""
    prompt = ChatPromptTemplate.from_template(
        "Provide a billing support response to the following query: {query}"
    )
    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content
    return {"response": response}

def handle_general(state: State) -> State:
    """Provide a general support response to the query."""
    prompt = ChatPromptTemplate.from_template(
        "Provide a general support response to the following query: {query}"
    )
    chain = prompt | llm
    response = chain.invoke({"query": state["query"]}).content
    return {"response": response}

def escalate(state: State) -> State:
    """Escalate the query to a human agent due to negative sentiment."""
    return {"response": "This query has been escalated to a human agent due to its negative sentiment."}

def route_query(state: State) -> str:
    """Route the query based on its sentiment and category."""
    if state["sentiment"] == "Negative":
        return "escalate"
    elif state["category"] == "Technical":
        return "handle_technical"
    elif state["category"] == "Billing":
        return "handle_billing"
    else:
        return "handle_general"

## Create and Configure the Graph

Here we set up the LangGraph, defining nodes and edges to create our customer support workflow.

In [6]:
# Check if llm (AzureChatOpenAI) has been imported and can generate a response

try:
    test_prompt = ChatPromptTemplate.from_template("Who is President of India!")
    chain = test_prompt | llm
    response = chain.invoke({})
    print("LLM import and chat established successfully.")
    print("Sample response:", response.content)
except Exception as e:
    print("LLM import or chat failed:", str(e))

LLM import and chat established successfully.
Sample response: As of my last knowledge update in October 2023, the President of India is Droupadi Murmu. She took office on July 25, 2022, and is the first tribal woman to hold the position. Please verify with up-to-date sources, as political positions can change.


In [9]:
# Create the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("categorize", categorize)
workflow.add_node("analyze_sentiment", analyze_sentiment)
workflow.add_node("handle_technical", handle_technical)
workflow.add_node("handle_billing", handle_billing)
workflow.add_node("handle_general", handle_general)
workflow.add_node("escalate", escalate)

# Add edges
workflow.add_edge("categorize", "analyze_sentiment")
workflow.add_conditional_edges(
    "analyze_sentiment",
    route_query,
    {
        "handle_technical": "handle_technical",
        "handle_billing": "handle_billing",
        "handle_general": "handle_general",
        "escalate": "escalate"
    }
)
workflow.add_edge("handle_technical", END)
workflow.add_edge("handle_billing", END)
workflow.add_edge("handle_general", END)
workflow.add_edge("escalate", END)

# Set entry point
workflow.set_entry_point("categorize")

# Compile the graph
app = workflow.compile()

In [10]:
# Check if the compiled LangGraph app exists and is ready
if app is not None:
    print("The LangGraph app has been created successfully.")
else:
    print("The LangGraph app has NOT been created.")

The LangGraph app has been created successfully.


## Visualize the Graph

This cell generates and displays a visual representation of our LangGraph workflow.

In [11]:
display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
            max_retries=5,
            retry_delay=2.0,
        )
    )
)

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 5 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

## Run Customer Support Function

This function processes a customer query through our LangGraph workflow.

In [12]:
def run_customer_support(query: str) -> Dict[str, str]:
    """Process a customer query through the LangGraph workflow.
    
    Args:
        query (str): The customer's query
        
    Returns:
        Dict[str, str]: A dictionary containing the query's category, sentiment, and response
    """
    results = app.invoke({"query": query})
    return {
        "category": results["category"],
        "sentiment": results["sentiment"],
        "response": results["response"]
    }

In [13]:
# Debug: Test the function with a sample query

test_queries = [
    "My internet connection keeps dropping. Can you help?",
    "I need help talking to chatGPT",
    "where can i find my receipt?",
    "What are your business hours?"
]
for q in test_queries:
    try:
        result = run_customer_support(q)
        print(f"Query: {q}")
        print(f"Category: {result['category']}")
        print(f"Sentiment: {result['sentiment']}")
        print(f"Response: {result['response']}")
        print("-" * 40)
    except Exception as e:
        print(f"Error processing query '{q}': {e}")

Query: My internet connection keeps dropping. Can you help?
Category: The query "My internet connection keeps dropping. Can you help?" falls into the **Technical** category.
Sentiment: Negative
Response: This query has been escalated to a human agent due to its negative sentiment.
----------------------------------------
Query: I need help talking to chatGPT
Category: The query "I need help talking to chatGPT" can be categorized as **General**.
Sentiment: Neutral
Response: Of course! I'm here to help you with that. If you're looking to have a conversation with ChatGPT, here are a few tips to get started:

1. **Be Clear and Specific**: Try to ask clear and specific questions. The more detail you provide, the better the response will be.

2. **Use Natural Language**: You can talk to ChatGPT just like you would with a person. Feel free to use complete sentences and express your thoughts.

3. **Ask Follow-Up Questions**: If you don’t get the answer you’re looking for, don’t hesitate to ask

## Test the Customer Support Agent

Let's test our customer support agent with a sample queries for each kind of query type.

In [14]:
# escalate

query = "My internet connection keeps dropping. Can you help?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

# handle_technical

query = "I need help talking to chatGPT"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

# handle_billing

query = "where can i find my receipt?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

# handle_general

query = "What are your business hours?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")


Query: My internet connection keeps dropping. Can you help?
Category: The query "My internet connection keeps dropping. Can you help?" falls into the **Technical** category.
Sentiment: Negative
Response: This query has been escalated to a human agent due to its negative sentiment.


Query: I need help talking to chatGPT
Category: The query "I need help talking to chatGPT" can be categorized as **General**.
Sentiment: Neutral
Response: Of course! I'm here to help you with any questions or concerns you have about using ChatGPT. Here are a few tips to get started:

1. **Be Clear and Specific**: When asking questions, try to be as clear and specific as possible. This helps ChatGPT understand what you're looking for.

2. **Ask Follow-Up Questions**: If the response you receive isn't quite what you were looking for, feel free to ask follow-up questions for clarification or more details.

3. **Explore Different Topics**: You can ask about a wide range of topics, from general knowledge and adv